In [2]:
import torch
import torch.nn as nn

from transformer.transformer_model import *

In [3]:
use_big_transformer = False

# Dummy data
src_vocab_size = 11
trg_vocab_size = 11
src_token_ids_batch = torch.randint(1, 10, size=(3, 2))
trg_token_ids_batch = torch.randint(1, 10, size=(3, 2))

transformer = Transformer(
    model_dimension=BIG_MODEL_DIMENSION if use_big_transformer else BASELINE_MODEL_DIMENSION,
    src_vocab_size=src_vocab_size,
    trg_vocab_size=trg_vocab_size,
    number_of_heads=BIG_MODEL_NUMBER_OF_HEADS if use_big_transformer else BASELINE_MODEL_NUMBER_OF_HEADS,
    number_of_layers=BIG_MODEL_NUMBER_OF_LAYERS if use_big_transformer else BASELINE_MODEL_NUMBER_OF_LAYERS,
    dropout_probability=BIG_MODEL_DROPOUT_PROB if use_big_transformer else BASELINE_MODEL_DROPOUT_PROB
)

# These 2 functions helped me figure out the 2 bugs I had:
# 1) I did not register positional encodings and thus they wouldn't be saved and later model-loading would fail
# 2) I had a bug with MHA (attention) in decoder, where both src and trg were referencing the same MHA object in mem
# It's a good practice to see whether the names, shapes and number of params make sense.
# e.g. I knew that the big transformer had ~175 M params and I verified that here.
analyze_state_dict_shapes_and_names(transformer)
print(f'Size of the {"big" if use_big_transformer else "baseline"} transformer = {count_parameters(transformer)}')

out = transformer(src_token_ids_batch, trg_token_ids_batch, src_mask=None, trg_mask=None)

.multi_headed_attention.out_projection_net.weight torch.Size([512, 512])
encoder.encoder_layers.0.multi_headed_attention.out_projection_net.bias torch.Size([512])
encoder.encoder_layers.0.pointwise_net.linear1.weight torch.Size([2048, 512])
encoder.encoder_layers.0.pointwise_net.linear1.bias torch.Size([2048])
encoder.encoder_layers.0.pointwise_net.linear2.weight torch.Size([512, 2048])
encoder.encoder_layers.0.pointwise_net.linear2.bias torch.Size([512])
encoder.encoder_layers.1.sublayers.0.norm.weight torch.Size([512])
encoder.encoder_layers.1.sublayers.0.norm.bias torch.Size([512])
encoder.encoder_layers.1.sublayers.1.norm.weight torch.Size([512])
encoder.encoder_layers.1.sublayers.1.norm.bias torch.Size([512])
encoder.encoder_layers.1.multi_headed_attention.qkv_nets.0.weight torch.Size([512, 512])
encoder.encoder_layers.1.multi_headed_attention.qkv_nets.0.bias torch.Size([512])
encoder.encoder_layers.1.multi_headed_attention.qkv_nets.1.weight torch.Size([512, 512])
encoder.encoder_